<div style='float: right'><img src='pic/slither.png'/></div>
## <div id='slither' />スリザーリンク

In [ ]:
import numpy as np
from itertools import product
from pulp import *
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = """\
33...3.
..2....
...21.3
23...02
0.03...
....0..
.3...33""".split()
nw, nh = len(data[0]), len(data)
M = (nw + 1) * (nh + 1) - 1
r2 = range(2)

### 問題
* 点の間を線でつなぎ、一つの輪を作ります
* 数字は、周囲の線の数に等しいこと
* 一つの点から出る線の数は0か2となること

### 考え方
* 線は無向ですが、有向として考えます
* 数字の3がある角を始点とします（必ず通ります）

### 変数
* vh：0:to left, 1:to right (1)
* vv：0:to down, 1:to up (2)
* vz：始点からの距離 (3)

### 制約
* 数字があれば、矢印の数に等しいこと (4)
* 矢印の向きは1つ (5)
* 始点以外は、矢印があれば距離が増やします (6)
* 始点の距離は0 (7)
* 距離は、全点数以下 (8)
* 入る矢印数は1以下 (9)
* 入る数と出る数は同じになること (10)

In [ ]:
m = LpProblem()
vh = np.array(addbinvars(nh+1, nw, 2)) # 0:to left, 1:to right (1)
vv = np.array(addbinvars(nh, nw+1, 2)) # 0:to down, 1:to up (2)
vz = np.array(addvars(nh+1, nw+1)) # (3)
def dirs(i, j, k):
    return ([vh[i,j-l,k^l] for l in r2 if 0 <= j-l < nw] +
            [vv[i-l,j,k^l] for l in r2 if 0 <= i-l < nh])
for i, j in product(range(nh), range(nw)):
    if data[i][j].isdigit():
        m += lpSum(vh[i+k,j,l] + vv[i,j+k,l] for l in r2 for k in r2) == int(data[i][j]) # (4)
        if data[i][j] == '3':
            fy, fx = i, j
m += vz[fy,fx] == 0 # (7)
for i, j in product(range(nh+1), range(nw)):
    m += lpSum(vh[i,j]) <= 1 # (5)
    m += vz[i,j] + 1 <= vz[i,j+1] + M * (1 - vh[i,j,0]) # (6)
    if (i, j) != (fy, fx):
        m += vz[i,j+1] + 1 <= vz[i,j] + M * (1 - vh[i,j,1]) # (6)
for i, j in product(range(nh), range(nw+1)):
    m += lpSum(vv[i,j]) <= 1 # (5)
    m += vz[i,j] + 1 <= vz[i+1,j] + M * (1 - vv[i,j,0]) # (6)
    if (i, j) != (fy, fx):
        m += vz[i+1,j] + 1 <= vz[i,j] + M * (1 - vv[i,j,1]) # (6)
for i, j in product(range(nh+1), range(nw+1)):
    m += vz[i,j] <= M # (8)
    din = dirs(i, j, 1)
    dout = dirs(i, j, 0)
    m += lpSum(din) <= 1 # (9)
    m += lpSum(din) == lpSum(dout) # (10)
%time m.solve()
rh = np.vectorize(value)(vh.sum(2))
rv = np.vectorize(value)(vv.sum(2))
for i in range(nh+1):
    print(' ', end='')
    for j in range(nw):
        print('- ' if rh[i,j] else '  ', end='')
    print()
    if i == nh: break
    for j in range(nw+1):
        print('|' if rv[i,j] else ' ', end='')
        if j < nh:
            print(data[i][j], end='')
    print()